# 6주차
## TNT 2019년 2학기 2부 발제

## K-means Clustering and Principal Component Analysis
이번 주차는 K-means와 PCA를 구현해보겠습니다. 

In [ ]:
# 저번에 안한 사람만
# socketio module 설치
# !pip install python-socketio

In [ ]:
import os
import sys
import numpy as np
import re

from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl

from IPython.display import HTML, display, clear_output

try:
    pyplot.rcParams["animation.html"] = "jshtml"
except ValueError:
    pyplot.rcParams["animation.html"] = "html5"

from scipy import optimize
from scipy.io import loadmat

sys.path.append('../src/')
from utils import Submit
import answer


#%load_ext autoreload
#%autoreload 2
%matplotlib inline

In [ ]:
submission = Submit()

In [ ]:
# board 연결이 필요할 경우만 실행
submission.connect()

## K-means Clustering
먼저 2D data에 대해 K-means 알고리즘을 적용해보겠습니다.

### 1.1 Implementing K-means
K-means 알고리즘은 비슷한 데이터끼리 cluster를 만듭니다. 예를 들어 $\{x^{(1)} , \cdots, x^{(m)}\}$ (where $x^{(i)} \in \mathbb{R}^n$) 과 같은 training set을 몇개의 밀도있는 cluster로 묶고 싶을 때 사용할 수 있습니다. K-means는 initial centroids 설정 이후, centroids와의 거리에 따른 cluster 부여, centroids update의 과정을 반복적으로 진행합니다.

```python
centroids = kMeansInitCentroids(X, K)
for i in range(iterations):
    # Cluster assignment step: 각각의 데이터에 가장 가까운 centroid를 할당.
    idx = findClosestCentroids(X, centroids)
    
    # Move centroid step: centroids에 속한 데이터들의 평균으로 새로운 centroids 위치 설정
    centroids = computeMeans(X, idx, K)
```

K-means 알고리즘은 항상 특정 결과로 수렴하지만, 그 결과는 최적점이 아닐 수도 있습니다. 이를 해결하기 위한 한 가지 방법은 서로 다른 초기값을 지정해 가장 낮은 cost를 갖는 모델을 선정하는 것입니다.


### 1.1.1 Finding closest centroids
K-means 알고리즘의 cluster assignment 단계에서 모든 데이터 $x^{(i)}$를 아래와 같은 조건에서 가장 가까운 centroid에 배정해야 합니다. 

$$c^{(i)} := j \quad \text{that minimizes} \quad \lvert\rvert x^{(i)} - \mu_j  \lvert\rvert^2, $$

$c^{(i)}$는 $x^{(i)}$와 가장 가까운 centroid의 index입니다. 그리고 $\mu_j$는 $j^{th}$ centroid의 값(또는 위치)입니다.
data matrix `X`와 현재 centroids의 위치를 입력으로 받아 각 데이터에 배정된 centroid index를 출력하는 함수를 만들어보세요. 

In [ ]:
def findClosestCentroids(X, centroids):
    """
    Computes the centroid memberships for every example.
    
    Parameters
    ----------
    X : array_like
        The dataset of size (m, n) where each row is a single example. 
        That is, we have m examples each of n dimensions.
        
    centroids : array_like
        The k-means centroids of size (K, n). K is the number
        of clusters, and n is the the data dimension.
    
    Returns
    -------
    idx : array_like
        A vector of size (m, ) which holds the centroids assignment for each
        example (row) in the dataset X.
    
    Instructions
    ------------
    Go over every example, find its closest centroid, and store
    the index inside `idx` at the appropriate location.
    Concretely, idx[i] should contain the index of the centroid
    closest to example i. Hence, it should be a value in the 
    range 0..K-1

    Note
    ----
    You can use a for-loop over the examples to compute this.
    """
    # Set K
    K = centroids.shape[0]
    idx = np.zeros(X.shape[0], dtype=int)

    # ====================== YOUR CODE HERE ======================

    for i,j in enumerate(X):
        distances = None
        idx[i] = None  
    
    # =============================================================
    return idx

In [ ]:
# Load an example dataset that we will be using위
data = loadmat(os.path.join('Data', 'ex7data2.mat'))
X = data['X']

# Select an initial set of centroids
K = 3   # 3 Centroids
initial_centroids = np.array([[3, 3], [6, 2], [8, 5]])

# Find the closest centroids for the examples using the initial_centroids
idx = findClosestCentroids(X, initial_centroids)

print('Closest centroids for the first 3 examples:')
print(idx[:3])
print('(the closest centroids should be 0, 2, 1 respectively)')

In [ ]:
submission.submit(findClosestCentroids)

### 1.1.2 Computing centroid means
각 데이터에 centroid가 배정되었다면, centroid의 위치를 데이터들의 평균으로 업데이트할 차례입니다.

$$ \mu_k := \frac{1}{\left| C_k\right|} \sum_{i \in C_k} x^{(i)}$$

$C_k$는 centroid $k$에 배정된 데이터들의 집합입니다. 예를 들어, 두 데이터 $x^{(3)}$와 $x^{(5)}$가 centroid $k = 2$에 배정되었다면, $\mu_2 = \frac{1}{2} \left( x^{(3)} + x^{(5)} \right)$로 업데이트 해야 합니다.centroid의 새로운 위치를 계산하는 `computeCentroids` 함수를 완성하세요. 당신이 초보자라면 loop 연산으로, 중급자라면 벡터 연산으로 계산해보세요. 설마 초급자겠어

In [ ]:
def computeCentroids(X, idx, K):
    """
    Returns the new centroids by computing the means of the data points
    assigned to each centroid.
    
    Parameters
    ----------
    X : array_like
        The datset where each row is a single data point. That is, it 
        is a matrix of size (m, n) where there are m datapoints each
        having n dimensions. 
    
    idx : array_like 
        A vector (size m) of centroid assignments (i.e. each entry in range [0 ... K-1])
        for each example.
    
    K : int
        Number of clusters
    
    Returns
    -------
    centroids : array_like
        A matrix of size (K, n) where each row is the mean of the data 
        points assigned to it.
    
    Instructions
    ------------
    Go over every centroid and compute mean of all points that
    belong to it. Concretely, the row vector centroids[i, :]
    should contain the mean of the data points assigned to
    cluster i.

    Note:
    -----
    You can use a for-loop over the centroids to compute this.
    """
    # Useful variables
    m, n = X.shape
    centroids = np.zeros((K, n))

    # ====================== YOUR CODE HERE ======================
    
    for i in range(K):
        centroids[i] = None
    
    # =============================================================
    return centroids

Once you have completed the code in `computeCentroids`, the following cell will run your code and output the centroids after the first step of K-means.

In [ ]:
# Compute means based on the closest centroids found in the previous part.
centroids = computeCentroids(X, idx, K)

print('Centroids computed after initial finding of closest centroids:')
print(centroids)
print('\nThe centroids should be')
print('   [ 2.428301 3.157924 ]')
print('   [ 5.813503 2.633656 ]')
print('   [ 7.119387 3.616684 ]')

*You should now submit your solutions.*

In [ ]:
submission.submit(computeCentroids)

## 1.2 K-means on example dataset
위의 두 함수 (`findClosestCentroids` and `computeCentroids`)를 성공적으로 완성했다면, 여러분들은 k-means 알고리즘에서 필요한 모든 것들을 만드셨습니다.다음으로 2D 데이터에 우리가 만든 알고리즘을 적용해보겠습니다.여러분이 만든 함수가 `runKmeans` 함수 안에서 동작할 것입니다. 아래의 그림과 같은 결과가 나오는 지 확인해보세요.

![](Figures/kmeans_result.png)

In [ ]:
# Load an example dataset
data = loadmat(os.path.join('Data', 'ex7data2.mat'))

# Settings for running K-Means
K = 3
max_iters = 10

# For consistency, here we set centroids to specific values
# but in practice you want to generate them automatically, such as by
# settings them to be random examples (as can be seen in
# kMeansInitCentroids).
initial_centroids = np.array([[3, 3], [6, 2], [8, 5]])


# Run K-Means algorithm. The 'true' at the end tells our function to plot
# the progress of K-Means
centroids, idx, anim = answer.runkMeans(X, initial_centroids,
                                        findClosestCentroids, 
                                        computeCentroids, 
                                        max_iters, True)
anim

## 2 Principal Component Analysis

### 2.1 Example Dataset
PCA를 통해 차원축소를 해보겠습니다. 먼저 간단한 2D 데이터에 적용해보겠습니다.

In [ ]:
# 데이터 불러오기
data = loadmat(os.path.join('Data', 'ex7data1.mat'))
X = data['X']

# 데이터 시각화
pyplot.plot(X[:, 0], X[:, 1], 'bo', ms=10, mec='k', mew=1)
pyplot.axis([0.5, 6.5, 2, 8])
pyplot.gca().set_aspect('equal')
pyplot.grid(False)

## 2.2 Implementing PCA
PCA는 아래와 같은 순서로 진행됩니다.
1. data의 공분산 구하기
2. SVD (`np.linalg.svd`)로 고유벡터 $U_1$, $U_2$, $\dots$, $U_n$ 계산하기.

먼저 아래와 같이 data의 공분산을 구하세요.
$$ \Sigma = \frac{1}{m} X^T X$$
$X$는 data matrix를, $m$은 데이터의 수를 의미합니다.

공분산을 계산한 뒤 SVD를 적용하세요. `numpy`를 사용해 `U, S, V = np.linalg.svd(Sigma)`와 같이 적용할 수 있습니다.

In [ ]:
def pca(X):
    """
    Run principal component analysis.
    
    Parameters
    ----------
    X : array_like
        The dataset to be used for computing PCA. It has dimensions (m x n)
        where m is the number of examples (observations) and n is 
        the number of features.
    
    Returns
    -------
    U : array_like
        The eigenvectors, representing the computed principal components
        of X. U has dimensions (n x n) where each column is a single 
        principal component.
    
    S : array_like
        A vector of size n, contaning the singular values for each
        principal component. Note this is the diagonal of the matrix we 
        mentioned in class.
    
    Instructions
    ------------
    You should first compute the covariance matrix. Then, you
    should use the "svd" function to compute the eigenvectors
    and eigenvalues of the covariance matrix. 

    Notes
    -----
    When computing the covariance matrix, remember to divide by m (the
    number of examples).
    """
    # Useful values
    m, n = X.shape

    # ====================== YOUR CODE HERE ======================
    
    Sigma = None
    U, S, V = None

    # ============================================================
    return U, S

PCA를 적용하기 전에 데이터를 normailze 해주는 것이 중요합니다. `featureNormalize` 함수로 normalize를 진행한 뒤 PCA를 적용해보세요.

![](Figures/pca_components.png)

In [ ]:
#  Before running PCA, it is important to first normalize X
X_norm, mu, sigma = answer.featureNormalize(X)

#  Run PCA
U, S = pca(X_norm)

#  Draw the eigenvectors centered at mean of data. These lines show the
#  directions of maximum variations in the dataset.
fig, ax = pyplot.subplots()
ax.plot(X[:, 0], X[:, 1], 'bo', ms=10, mec='k', mew=0.25)

for i in range(2):
    ax.arrow(mu[0], mu[1], 1.5 * S[i]*U[0, i], 1.5 * S[i]*U[1, i],
             head_width=0.25, head_length=0.2, fc='k', ec='k', lw=2, zorder=1000)

ax.axis([0.5, 6.5, 2, 8])
ax.set_aspect('equal')
ax.grid(False)

print('Top eigenvector: U[:, 0] = [{:.6f} {:.6f}]'.format(U[0, 0], U[1, 0]))
print(' (you should expect to see [-0.707107 -0.707107])')

*You should now submit your solutions.*

In [ ]:
submission.submit(pca)

## 2.3 Dimensionality Reduction with PCA
주성분을 계산한 뒤, 여러분들의 데이터를 더 낮은 차원으로 투영할 수 있습니다.
$x^{(i)} \rightarrow z^{(i)}$ (e.g., projecting the data from 2D to 1D)
여러분들이 학습 알고리즘에 축소된 데이터를 적용해보세요. 더 빠른 진행을 확인할 수 있습니다.

### 2.3.1 Projecting the data onto the principal components
`projectData` 함수를 완성하세요. `X`는 dataset, `U`는 주성분, `K`는 축소할 차원을 의미합니다. `Ureduce = U[:, :K]`의 방법으로 지정된 K개 선정하세요.

In [ ]:
def projectData(X, U, K):
    """
    Computes the reduced data representation when projecting only 
    on to the top K eigenvectors.
    
    Parameters
    ----------
    X : array_like
        The input dataset of shape (m x n). The dataset is assumed to be 
        normalized.
    
    U : array_like
        The computed eigenvectors using PCA. This is a matrix of 
        shape (n x n). Each column in the matrix represents a single
        eigenvector (or a single principal component).
    
    K : int
        Number of dimensions to project onto. Must be smaller than n.
    
    Returns
    -------
    Z : array_like
        The projects of the dataset onto the top K eigenvectors. 
        This will be a matrix of shape (m x k).
    
    Instructions
    ------------
    Compute the projection of the data using only the top K 
    eigenvectors in U (first K columns). 
    For the i-th example X[i,:], the projection on to the k-th 
    eigenvector is given as follows:
    
        x = X[i, :]
        projection_k = np.dot(x,  U[:, k])

    """

    # ====================== YOUR CODE HERE ======================

    #Z = np.dot(X, U[:,:K])
    Ureduce = None
    Z = None
    
    # =============================================================
    return Z

In [ ]:
#  Project the data onto K = 1 dimension
K = 1
Z = projectData(X_norm, U, K)
print('Projection of the first example: {:.6f}'.format(Z[0, 0]))
print('(this value should be about    : 1.481274)')

In [ ]:
submission.submit(projectData)

### 2.3.2 Reconstructing an approximation of the data
차원축소한 데이터를 다시 복원할 수 있어야 합니다. `recoverData` 함수를 완성해 원래 차원으로 복원했을 때의 추정값 `Xrec`을 구하세요.

In [ ]:
def recoverData(Z, U, K):
    """
    Recovers an approximation of the original data when using the 
    projected data.
    
    Parameters
    ----------
    Z : array_like
        The reduced data after applying PCA. This is a matrix
        of shape (m x K).
    
    U : array_like
        The eigenvectors (principal components) computed by PCA.
        This is a matrix of shape (n x n) where each column represents
        a single eigenvector.
    
    K : int
        The number of principal components retained
        (should be less than n).
    
    Returns
    -------
    X_rec : array_like
        The recovered data after transformation back to the original 
        dataset space. This is a matrix of shape (m x n), where m is 
        the number of examples and n is the dimensions (number of
        features) of original datatset.
    
    Instructions
    ------------
    Compute the approximation of the data by projecting back
    onto the original space using the top K eigenvectors in U.
    For the i-th example Z[i,:], the (approximate)
    recovered data for dimension j is given as follows:

        v = Z[i, :]
        recovered_j = np.dot(v, U[j, :K])

    Notice that U[j, :K] is a vector of size K.
    """
    # You need to return the following variables correctly.
    X_rec = np.zeros((Z.shape[0], U.shape[0]))

    # ====================== YOUR CODE HERE ======================

    X_rec = None

    # =============================================================
    return X_rec

Once you have completed the code in `recoverData`, the following cell will recover an approximation of the first example and you should see a value of about `[-1.047 -1.047]`. The code will then plot the data in this reduced dimension space. This will show you what the data looks like when using only the corresponding eigenvectors to reconstruct it. An example of what you should get for PCA projection is shown in this figure: 

![](Figures/pca_reconstruction.png)

In the figure above, the original data points are indicated with the blue circles, while the projected data points are indicated with the red circles. The projection effectively only retains the information in the direction given by $U_1$. The dotted lines show the distance from the data points in original space to the projected space. Those dotted lines represent the error measure due to PCA projection.

In [ ]:
X_rec  = recoverData(Z, U, K)
print('Approximation of the first example: [{:.6f} {:.6f}]'.format(X_rec[0, 0], X_rec[0, 1]))
print('       (this value should be about  [-1.047419 -1.047419])')

#  Plot the normalized dataset (returned from featureNormalize)
fig, ax = pyplot.subplots(figsize=(5, 5))
ax.plot(X_norm[:, 0], X_norm[:, 1], 'bo', ms=8, mec='b', mew=0.5)
ax.set_aspect('equal')
ax.grid(False)
pyplot.axis([-3, 2.75, -3, 2.75])

# Draw lines connecting the projected points to the original points
ax.plot(X_rec[:, 0], X_rec[:, 1], 'ro', mec='r', mew=2, mfc='none')
for xnorm, xrec in zip(X_norm, X_rec):
    ax.plot([xnorm[0], xrec[0]], [xnorm[1], xrec[1]], '--k', lw=1)

*You should now submit your solutions.*

In [ ]:
submission.submit(recoverData)